## Machine Learning Operations (MLOps)

In [1]:
from IPython.core.display import HTML

def load_css():
    styles = open("css/custom.css", "r").read()
    return HTML(f"<style>{styles}</style>")

load_css()

## Importando os pacotes do projeto
Vamos centralizar nesta célula a importação de todos os pacotes que iremos utilizar neste notebook

In [2]:
from pathlib import Path
import sys 
import os

current_path = Path(os.getcwd())
parent_path = current_path.parent.absolute()
sys.path.append(str(parent_path))

import json 
import folium
import numpy as np
from sklearn.cluster import KMeans
import random
import brazilcep
from geopy.geocoders import Nominatim
import geopy.distance
from yellowbrick.cluster import KElbowVisualizer
from joblib import dump, load
import json
import configparser
import matplotlib.pyplot as plt
from matplotlib import gridspec
from commons.utils import plot_points, get_delivery_coordinates, load_training_data, prepare_points, prepare_input_points, add_clusters_to_markers, get_colors, enrich_points_with_cluster_info, generate_sample_points, add_sample_points_to_markers, get_drift_params
from mlflow.models import infer_signature
from mlflow import MlflowClient
import mlflow
import json
import requests
import boto3
import warnings
warnings.filterwarnings('ignore')

<div class="custom-slide">
    <div class="hands-on">
        Continuando nossa implementação
    </div>
</div>

In [3]:
os.environ['AWS_ACCESS_KEY_ID'] = 'user'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'password'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'

mlflow.set_tracking_uri("http://localhost:5000")

## Usando o modelo em modo batch

Podemos carregar e usar nosso modelo para fazer predições em modo batch, sem a necessidade de um endpoint para inferência.

Primeiramente, vamos criar uma função para retornar as informações do modelo do MLFlow

In [4]:
def get_model_uri_by_name_and_alias(model_name, alias):
    
    # Initialize MLflow client
    client = mlflow.tracking.MlflowClient()
    
    # Get the model version details by alias
    model_version_details = client.get_model_version_by_alias(name=model_name, alias=alias)
    
    # Construct the model URI
    model_uri = f"models:/{model_name}/{model_version_details.version}"

    return model_uri, model_version_details.run_id, model_version_details.version

Podemos recuperar as informações do nosso modelo da seguinte forma

In [5]:
# Example usage
model_name = 'prod.bootcamp.kmeans-clustering'
alias = 'champion'
model_uri, run_id, version = get_model_uri_by_name_and_alias(model_name, alias)

Já com as informações de referência do modelo, podemos carregá-lo

In [6]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri)
#logged_model = f"runs:/{run_id}/model"

# Access the underlying custom model
custom_model_instance = loaded_model._model_impl.python_model

# Predict on a Pandas DataFrame.
#import pandas as pd
#loaded_model.predict(pd.DataFrame(data))

E usar o modelo

In [7]:
data = {
    'method': 'predict',
    'data': [-15.65246980577859, -47.7609277270516]
} 
loaded_model.predict(data)

{'is_region_covered': True,
 'closest_center': {'id': 2,
  'distance_in_km': 3.16,
  'lat': -15.757037977498,
  'lng': -47.771612075032984}}

In [8]:
data = {
    'method': 'get_cluster_centroids',
    'data': None
} 
loaded_model.predict(data)

[{'lat': -15.623188710465486, 'lng': -47.65238397469197, 'cluster': 1},
 {'lat': -15.649745950686997, 'lng': -47.7902749412903, 'cluster': 2},
 {'lat': -15.757037977498, 'lng': -47.771612075032984, 'cluster': 3},
 {'lat': -15.890221773669836, 'lng': -47.497399544416034, 'cluster': 4},
 {'lat': -15.603417375334981, 'lng': -47.917044479085696, 'cluster': 5},
 {'lat': -15.65940145501782, 'lng': -47.837297176334864, 'cluster': 6}]

In [9]:
data = {
    'method': 'get_model_version',
    'data': None
} 
loaded_model.predict(data)

{'version': '25',
 'run_id': 'ea9146b52ee44f828cb0aaf635ee144d',
 'model_uri': 'models:/prod.bootcamp.kmeans-clustering/25'}

## Servindo o modelo via API

https://mlflow.org/docs/latest/deployment/deploy-model-locally.html

Basta acessar o terminar e digitar 

`export MLFLOW_TRACKING_URI=http://localhost:5000`<br>
`export AWS_ACCESS_KEY_ID=user`<br>
`export AWS_SECRET_ACCESS_KEY=password`<br>
`export MLFLOW_S3_ENDPOINT_URL=http://localhost:9000`<br>
`mlflow models serve -m "models:/prod.bootcamp.kmeans-clustering@champion" -p8001  --no-conda`

In [10]:
#model = mlflow.pyfunc.load_model("runs:/{}/model".format(run_id))
#model.serve(port=8001)


In [13]:
payload = json.dumps(
    {
        'inputs': {
            'method': 'get_model_version',
            'data': None
        }
    }
)

response = requests.post(
    url=f"http://localhost:8001/invocations",
    data=payload,
    headers={"Content-Type": "application/json"},
)
print(response.json())

{'predictions': {'version': '25', 'run_id': 'ea9146b52ee44f828cb0aaf635ee144d', 'model_uri': 'models:/prod.bootcamp.kmeans-clustering/25'}}


In [14]:
payload = json.dumps(
    {
        'inputs': {
            'method': 'get_cluster_centroids',
            'data': None
        }
    }
)

response = requests.post(
    url=f"http://localhost:8001/invocations",
    data=payload,
    headers={"Content-Type": "application/json"},
)
print(response.json())

{'predictions': [{'lat': -15.623188710465486, 'lng': -47.65238397469197, 'cluster': 1}, {'lat': -15.649745950686997, 'lng': -47.7902749412903, 'cluster': 2}, {'lat': -15.757037977498, 'lng': -47.771612075032984, 'cluster': 3}, {'lat': -15.890221773669836, 'lng': -47.497399544416034, 'cluster': 4}, {'lat': -15.603417375334981, 'lng': -47.917044479085696, 'cluster': 5}, {'lat': -15.65940145501782, 'lng': -47.837297176334864, 'cluster': 6}]}


In [15]:
payload = json.dumps(
    {
        'inputs': {
            'method': 'predict',
            'data': [-15.913988770795076, -47.60220527168726]
        }
    }
)

response = requests.post(
    url=f"http://localhost:8001/invocations",
    data=payload,
    headers={"Content-Type": "application/json"},
)
print(response.json())

{'predictions': {'is_region_covered': False, 'closest_center': {'id': 4, 'distance_in_km': 11.53, 'lat': -15.603417375334981, 'lng': -47.917044479085696}}}


<div class="custom-slide">
    <h1>Conclusão</h1>
    <p>
       Parabéns por ter acompanhado o curso até aqui! Realmente fizemos muitas coisas interessantes:
    </p>
    <ul>
        <li>Aulas 1, 2 e 3: Exploramos o problema e criamos um modelo completo de ML</li>
        <li>Aula 4: Analisamos o erro do modelo e registramos os resultados para futura análise de model drift</li>
        <li>Aula 5: Aprendemos como utilizar um model registry (MLFlow)</li>
        <li>Aula 6: Aprendemos como criar um modelo customizado, incorporando algoritmos de ML e regras de negócio. </li>
        <li>Aula 7: Aprendemos como implementar os pipelines de inferência (batch e online)</li>
    </ul>
    <p>
       Nas próximas aulas, iremos sair um pouco do ambiente do jupyter notebook para realizar as seguintes tarefas:
    </p>
    <ul>
        <li>Automatizar o pipeline de (re)treino do modelo</li>
        <li>Implementar um processo de CI/CD para promover nosso modelo para o ambiente de produção</li>
        <li>Implementar nossa própria API para servir o modelo</li>
        <li>Analisar o model drift</li>
    </ul>
    <p>
       Vamos em frente para a parte final do curso 🚀
    </p>
</div>

<div class="custom-slide">
<h1>Discutindo sobre alta escalabilidade</h1>
    <img src="images/mlflow-local-serving.png" alt="Sample Image" style="width:100%;border-radius:10px;">
    <span class="image-ref">Source: https://mlflow.org/docs/latest/deployment/deploy-model-locally.html</span>
</div>

<div class="custom-slide">
    <h1>Arquitetura de referência</h1>
    <img src="images/mlflow-prod-deployment.png" alt="Sample Image" style="width:100%;border-radius:10px;">
    <span class="image-ref">Source: https://mlflow.org/docs/latest/deployment/index.html#id1</span>
</div>